In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
#!pip install opencv-python

In [6]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GroupKFold


from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold

from collections import defaultdict

In [8]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
import logging
from typing import Dict, IO, Iterator, Sequence, Union
from pyteomics import mgf
import pyteomics
from spectrum_utils.spectrum import MsmsSpectrum
import matplotlib.pyplot as plt
import spectrum_utils.plot as sup
import spectrum_utils as su
import numpy as np
from src.load_data import LoadData
from spectrum_utils import plot
from src.preprocessor import Preprocessor
from src.ml_model import MlModel
from src.tanimoto import Tanimoto
from sklearn.model_selection import train_test_split
from src.plotting import Plotting
import pickle
from src.cosine_distance import CosineDistance

In [7]:
# method 2
def get_bm_scaffold(smiles):
    try:
        scaffold = Chem.MolToSmiles(MurckoScaffold.MakeScaffoldGeneric(mol=Chem.MolFromSmiles(smiles)))
    except Exception:
        print("Raise AtomValenceException, return basic Murcko Scaffold")
        scaffold = smiles
    return scaffold


In [10]:
mgf_path = r"C:\Users\ramir\Downloads\ALL_GNPS_NO_PROPOGATED_wb.mgf"

## Open data

In [11]:
all_spectrums_original = LoadData.get_all_spectrums(mgf_path,100)

100%|██████████| 100/100 [00:08<00:00, 11.47it/s]


In [13]:
smiles_temp = all_spectrums_original[0].params['smiles']

In [14]:
smiles_temp

'CC(C)CC1NC(=O)C(C)NC(=O)C(=C)N(C)C(=O)CCC(NC(=O)C(C)C(NC(=O)C(CCCNC(N)=N)NC(=O)C(C)C(NC1=O)C(O)=O)\\C=C\\C(\\C)=C\\C(C)C(O)Cc1ccccc1)C(O)=O'

In [17]:
bms = [get_bm_scaffold(s.params['smiles']) for s in all_spectrums_original]

In [22]:
# Count the appearances of unique values
unique_values, counts = np.unique(bms, return_counts=True)

In [29]:
train_split=0.6
val_split= 0.2
test_split=0.2

In [32]:
train_index= int(train_split*(len(unique_values)))
val_index= train_index + int(val_split*(len(unique_values)))

In [33]:
train_index, val_index

(41, 54)

In [34]:
train_bms= unique_values[0:train_index]
val_bms = unique_values[train_index:val_index]
test_bms=unique_values[val_index:]

In [43]:
for s in all_spectrums_original:
    s.murcko_scaffold = get_bm_scaffold(s.params['smiles'])

In [44]:
import random
@staticmethod
def train_val_test_split_bms(spectrums, val_split=0.2, test_split=0.2):
    # get the murcko scaffold
    bms = [s.murcko_scaffold for s in spectrums]
    
    # count the unique elements
    unique_values, counts = np.unique(bms, return_counts=True)
    
    # randomize
    random.shuffle(unique_values)
    
    # get indexes
    train_index= int(train_split*(len(unique_values)))
    val_index= train_index + int(val_split*(len(unique_values)))
    
    # get elements
    train_bms= unique_values[0:train_index]
    val_bms = unique_values[train_index:val_index]
    test_bms=unique_values[val_index:]
    
    # get data
    spectrums_train = [s for s in spectrums if s.murcko_scaffold in train_bms]
    spectrums_val = [s for s in spectrums if s.murcko_scaffold in val_bms]
    spectrums_test = [s for s in spectrums if s.murcko_scaffold in test_bms]
    return spectrums_train, spectrums_val, spectrums_test
    

In [46]:
spectrums_train, spectrums_val, spectrums_test = train_val_test_split_bms(all_spectrums_original)

In [48]:
bms_train = [s.murcko_scaffold for s in spectrums_train]
bms_val = [s.murcko_scaffold for s in spectrums_val]
bms_test = [s.murcko_scaffold for s in spectrums_test]


In [50]:
[b for b in bms_train if b in bms_val]

[]

In [51]:
[b for b in bms_train if b in bms_test]

[]

In [52]:
[b for b in bms_val if b in bms_test]

[]